# `ULO`  

#### GENERAL FRAMEWORK 

* discrete pointwise minimum of <i>pieces</i> with common <i>functional constraints</i>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = "retina"
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
import cvxpy as cp
clear_output()
rs_seed = 120824#130824
np.random.seed(rs_seed)
import time
import itertools
import pandas as pd

<b>Problem</b>

$\omega \in (0,1]$

$$ \min_{x \in \mathcal{X}}\,\frac{\tau}{2}||x||^2 + \omega \cdot \max_{\tilde{i} \in [n]} h^{(\tilde{i})}(x) + (1-\omega) \cdot \min_{i \in [n]} h^{(i)}(x)$$
$$\max_{j \in [m]}\,c^{(j)}(x) \leq 0$$

In [2]:
## parameters 
TOL_REL_DEF = 0
TOL_ABS_DEF = 0
TOL_SIGNIF_CSTR_DEF = 1e-3
MAX_ITER_DEF = 1e3
MAX_TIME_ABS_DEF = 240
verb_def = False

name = 'nominal_experiment'

In [3]:
'''
@comment
'''
def RAM_OPO(list_q_h,list_q_c,list_kappa_h,list_kappa_c,omega=0,M=1e4,tau=0,init=0,rho=1e-3,domain_size=10,extra_verb=verb_def,\
        MAX_TIME=MAX_TIME_ABS_DEF,\
        MAX_ITER=MAX_ITER_DEF,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF):
    
    m = list_q_c.shape[0]
    n,d = list_q_h.shape
    
    ###### EVAL TOOLS
    F = lambda u,eta: tau/2*sum(u**2) + M*eta + omega*np.max(list_q_h@u+list_kappa_h) + (1-omega)*np.min(list_q_h@u+list_kappa_h)
    
    def active(u,eta):
        values = list_q_h@u + list_kappa_h
        return np.where(values<=min(values)+rho)[0]

    guarantees = []

    ###### INIT
    k = 0
    H = []
    F_check = -np.inf
    F_hat = np.inf
    condition = True
    to_check = np.arange(n)
    to_check = np.setdiff1d(to_check,[init])
    to_check = np.concatenate((np.array([init]),to_check))
    np.random.shuffle(to_check)
    selectable = [init]
    x_out = None
    
    ###### CVX
    eta_cvx,u_cvx = cp.Variable(1),cp.Variable(d)
    q_h_param,kappa_h_param = cp.Parameter(d),cp.Parameter(1)
    X_basic = [cp.sum(u_cvx)==domain_size,u_cvx>=0,eta_cvx>=0]
    if tau>0:
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) + tau/2*cp.sum_squares(u_cvx)
    else: 
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) 
    prob = cp.Problem(cp.Minimize(obj_cvx),X_basic+[list_q_c@u_cvx+list_kappa_c-eta_cvx[0]<=0])
    
    ###### SOLVE
    if extra_verb:
        print('=== NEW RAM SOLVE ===')
        print(' ')
        print('-> i hat = '+str(init))
        print(' ')

    t_ref = time.time()

    ## loop
    while condition:   
        
        # -> upper-models
        R,P = np.inf,np.inf
        while len(selectable)>0:
            i_bar = np.random.choice(selectable,replace=False,size=1)[0]
            q_h_param.value,kappa_h_param.value = list_q_h[i_bar],np.array([list_kappa_h[i_bar]])
            if tau>0:
                prob.solve(solver=cp.MOSEK,warm_start=True)
            else:
                prob.solve(solver=cp.SCIPY, scipy_options={"method": "highs"},warm_start=True)
            H.append(i_bar)
            if prob.value<R:
                u_bar,eta_bar = u_cvx.value,eta_cvx.value[0]
                R = F(u_bar,eta_bar)
                P = prob.value
                selectable = np.setdiff1d(active(u_bar,eta_bar),H) 
            else:
                selectable = np.setdiff1d(selectable,[i_bar])
                
        to_check = np.setdiff1d(to_check,H)
        np.random.shuffle(to_check)
        if len(to_check)>0:
            selectable = [to_check[0]]
                
        # update of upper-bound  
        if R<F_hat:
            i_out = i_bar
            F_hat = R
            print('t = '+str(time.time()-t_ref))
            print('F_hat <= '+str(R)+' for i_bar = '+str(i_bar))
            u_out,eta_out = u_bar,eta_bar
    
        guarantees.append((time.time()-t_ref,F_hat,F_check))
        
        # iteration counter increments
        k += 1
        
        # optimality or exit check
        condition = (k<MAX_ITER) and len(to_check)>0 and time.time()-t_ref<MAX_TIME
    
    print(' ')
    print('=== total time: '+str(time.time()-t_ref)+' ===')
    print('|H|/n = '+str((n-len(to_check))/n))


    if extra_verb:

        if k==MAX_ITER:
            print('/!\ EXIT: max. #iters. reached /!\ ')
        elif time.time()-t_ref>=MAX_TIME:
            print('/!\ EXIT: search time over... /!\ ')
        print(' -> final F = '+str(F(u_out,eta_out)))
    print(' ')
        
    return guarantees,u_out,eta_out

In [4]:
'''
@comment
'''
def ULO_OPO(list_q_h,list_q_c,list_kappa_h,list_kappa_c,omega=0,M=1e4,tau=0,init=0,rho=1e-3,domain_size=10,extra_verb=verb_def,\
        MAX_TIME=MAX_TIME_ABS_DEF,\
        MAX_ITER=MAX_ITER_DEF,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF):
    
    m = list_q_c.shape[0]
    n,d = list_q_h.shape
    
    ###### EVAL TOOLS
    F = lambda u,eta: tau/2*sum(u**2) + M*eta + omega*np.max(list_q_h@u+list_kappa_h) + (1-omega)*np.min(list_q_h@u+list_kappa_h)
    
    def active(u,eta):
        values = list_q_h@u + list_kappa_h
        return np.where(values<=min(values)+rho)[0]
    
    def support(u,eta):
        values = list_q_c@u + list_kappa_c - eta
        return np.where(values>=-TOL_SIGNIF_CSTR)[0]
    
    ###### STORAGE purposes
    cached_minimizers = {}
    for i in range(n):
        cached_minimizers[i] = (None,None)
    guarantees = []

    ###### INIT
    k = 0
    S = []
    H = []
    F_check = -np.inf
    F_hat = np.inf
    condition = True
    selectable = [init]
    not_converged = True
    x_out = None
    
    ###### CVX
    eta_cvx,u_cvx = cp.Variable(1),cp.Variable(d)
    q_h_param,kappa_h_param = cp.Parameter(d),cp.Parameter(1)
    X_basic = [cp.sum(u_cvx)==domain_size,u_cvx>=0,eta_cvx>=0]
    if tau>0:
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) + tau/2*cp.sum_squares(u_cvx)
    else: 
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) 
    prob = cp.Problem(cp.Minimize(obj_cvx),X_basic+[list_q_c@u_cvx+list_kappa_c-eta_cvx[0]<=0])
    
    ###### SOLVE
    if extra_verb:
        print('=== NEW ULO SOLVE ===')
        print(' ')
        print('-> i hat = '+str(init))
        print(' ')

    t_ref = time.time()

    ## loop
    while condition:   
        
        # -> upper-models
        R,P = np.inf,np.inf
        while len(selectable)>0:
            i_bar = np.random.choice(selectable,replace=False,size=1)[0]
            q_h_param.value,kappa_h_param.value = list_q_h[i_bar],np.array([list_kappa_h[i_bar]])
            if tau>0:
                prob.solve(solver=cp.MOSEK,warm_start=True)
            else:
                prob.solve(solver=cp.SCIPY, scipy_options={"method": "highs"},warm_start=True)
            H.append(i_bar)
            if prob.value<R:
                u_bar,eta_bar = u_cvx.value,eta_cvx.value[0]
                R = F(u_bar,eta_bar)
                P = prob.value
                selectable = np.setdiff1d(active(u_bar,eta_bar),H) 
            else:
                selectable = np.setdiff1d(selectable,[i_bar])
                
        # update of upper-bound  
        if P<F_hat:
            print('t = '+str(time.time()-t_ref))
            print('F_hat: '+str(R)+' for i_bar = '+str(i_bar))
            i_out = i_bar
            F_hat = R
            u_out,eta_out = u_bar,eta_bar
    
        guarantees.append((time.time()-t_ref,F_hat,F_check))
                            
        # -> lower-models
        new_S = list(np.setdiff1d(support(u_bar,eta_bar),S))
        S += new_S
        print('|S_'+str(k+1)+'| = '+str(len(S)))
        relaxed_prob = cp.Problem(cp.Minimize(obj_cvx),X_basic+[list_q_c[S]@u_cvx+list_kappa_c[S]-eta_cvx[0]<=0])
        new_mod_val,i_hat= F_hat,i_out
        restart_set = np.setdiff1d(np.arange(n),H)
        for i in restart_set:
            q_h_param.value,kappa_h_param.value = list_q_h[i],np.array([list_kappa_h[i]])
            if cached_minimizers[i][0] is not None:
                u_cvx.value,eta_cvx.value = cached_minimizers[i][0],cached_minimizers[i][1]
            if tau>0:
                relaxed_prob.solve(solver=cp.MOSEK,warm_start=True)
            else:
                relaxed_prob.solve(solver=cp.SCIPY, scipy_options={"method": "highs"},warm_start=True)
            cached_minimizers[i] = (u_cvx.value,eta_cvx.value)
            if relaxed_prob.value<=new_mod_val:
                new_mod_val = relaxed_prob.value
                i_hat = i
        print('i hat = '+str(i_hat))

        # update of lower-bound (and eventually retraced upper-bound)
        F_check = max(F_check,new_mod_val)
    
        guarantees.append((time.time()-t_ref,F_hat,F_check))
        
        not_converged = (F_hat-F_check)> max(TOL_ABS,max(1,abs(F_hat))*TOL_REL)

        selectable = [i_hat]
          
        if extra_verb:  
            print('t = '+str(time.time()-t_ref))
            print('F_hat: '+str(np.round(F_hat,4))+' & F_check: '+str(np.round(F_check,4))+'| GAP: '+str(max(0,F_hat-F_check))+' || target: '+str(max(TOL_ABS,max(1,abs(F_hat))*TOL_REL)))
        
        # iteration counter increments
        k += 1
        
        # optimality or exit check
        condition = (k<MAX_ITER) and not_converged and time.time()-t_ref<MAX_TIME
    
    print(' ')
    print('=== total time: '+str(time.time()-t_ref)+' ===')
    print('|H|/n = '+str(len(H)/n))


    if extra_verb:

        if k==MAX_ITER:
            print('/!\ EXIT: max. #iters. reached /!\ ')
        elif not_converged==False:
            print('/!\ EXIT: tolerated accuracy reached /!\ ') 
        elif time.time()-t_ref>=MAX_TIME:
            print('/!\ EXIT: search time over... /!\ ')
        print(' -> final F = '+str(F(u_out,eta_out)))
    print(' ')
        
    return guarantees,u_out,eta_out

In [5]:
'''
@comment
'''
def ES_OPO(list_q_h,list_q_c,list_kappa_h,list_kappa_c,omega=0,M=1e4,tau=0,init=0,domain_size=10,extra_verb=verb_def,\
        MAX_TIME=MAX_TIME_ABS_DEF,\
        MAX_ITER=MAX_ITER_DEF,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF):
    
    m = list_q_c.shape[0]
    n,d = list_q_h.shape
    
    ###### EVAL TOOLS
    F = lambda u,eta: tau/2*sum(u**2) + M*eta + omega*np.max(list_q_h@u+list_kappa_h) + (1-omega)*np.min(list_q_h@u+list_kappa_h)
    
    every_time_val = []
    
    ###### INIT
    k = 0
    F_hat = np.inf
    condition = True
    to_check = np.arange(n-1)
    np.random.shuffle(to_check)
    to_check = np.concatenate((np.array([init]),to_check))
    eta_out,u_out = None,None
    
    ###### CVX
    eta_cvx,u_cvx = cp.Variable(1),cp.Variable(d)
    q_h_param,kappa_h_param = cp.Parameter(d),cp.Parameter(1)
    X_basic = [cp.sum(u_cvx)==domain_size,u_cvx>=0,eta_cvx>=0]
    if tau>0:
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) + tau/2*cp.sum_squares(u_cvx)
    else: 
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) 
    prob = cp.Problem(cp.Minimize(obj_cvx),X_basic+[list_q_c@u_cvx+list_kappa_c-eta_cvx[0]<=0])
    
    ###### SOLVE
    if extra_verb:
        print('=== NEW ES SOLVE ===')
        print(' ')
        print('-> i hat = '+str(init))
        print(' ')

    t_ref = time.time()

    ## loop
    while condition:   
        
        # -> upper-models
        i_bar = to_check[0]
        q_h_param.value,kappa_h_param.value = list_q_h[i_bar],np.array([list_kappa_h[i_bar]])
        if tau>0:
            prob.solve(solver=cp.MOSEK,warm_start=True)
        else:
            prob.solve(solver=cp.SCIPY, scipy_options={"method": "highs"},warm_start=True)
        u_bar,eta_bar = u_cvx.value,eta_cvx.value[0]
        R = F(u_bar,eta_bar)
        
        every_time_val.append((time.time()-t_ref,R))

        to_check = to_check[1:]

        # update of upper-bound  
        if R<F_hat:
            i_out = i_bar
            F_hat = R
            print('t = '+str(time.time()-t_ref))
            print('F_hat <= '+str(R)+' for i_bar = '+str(i_bar))
            u_out,eta_out = u_bar,eta_bar
    
        # iteration counter increments
        k += 1
        
        # optimality or exit check
        condition = (k<MAX_ITER) and len(to_check)>0 and time.time()-t_ref<MAX_TIME
    
    print(' ')
    print('=== total time: '+str(time.time()-t_ref)+' ===')
    print('|H|/n = '+str((n-len(to_check))/n))

    if extra_verb:

        if k==MAX_ITER:
            print('/!\ EXIT: max. #iters. reached /!\ ')
        elif time.time()-t_ref>=MAX_TIME:
            print('/!\ EXIT: search time over... /!\ ')
        print(' -> final F = '+str(F(u_out,eta_out)))
    print(' ')
        
    return every_time_val,u_out,eta_out

### Scenarios Sampling

In [6]:
## GEOMETRY CONTROL CELL ##

dimension_global = int(100)

if name=='bign_experiment':
    m_global = int(1e3) 
    n_global = int(2e3) 

else:
    m_global = int(1e4) 
    n_global = int(5e2) 

timing = 240

ds = 10

# params
rel_impact = .15
cardinality_low,cardinality_high = int(max(1,dimension_global/10)),int(dimension_global)
ps = [1/10,1/10,3/5,1/10,1/10]

M_true = 5e4 # 1e4

omega_range = [1/2]

list_RESULTS = []
list_algorithm = []
list_rep = []
list_prob = []
list_omegas = []
list_cuts = []

num_probs = 1
num_starts = 15

maxit = np.inf

verb_ = True

tau_def = 0

cut_range = [1,5,1e-1]

In [7]:
for prob_id in range(num_probs):

    # profits
    PI_nominal = np.random.randint(0,15,dimension_global)
    PI = np.zeros((n_global,dimension_global))
    for i in range(n_global):
        PI[i] = np.maximum(0,PI_nominal + 10*np.random.choice([-2,-1,0,1,2],\
                                                replace=True,size=dimension_global,p=ps))

    # fixed costs
    distrib_PI = np.sum(PI,1)
    PHI = np.random.uniform(0,max(distrib_PI)-min(distrib_PI),n_global)*rel_impact

    importance_nominal = np.argsort(PI_nominal)

    ## investments constraints
    weights = np.array([float(elem)**5 for elem in np.arange(dimension_global)])
    weights /= np.sum(weights)
    MU,TAU = np.zeros((m_global,dimension_global)),np.zeros(m_global)
    for j in range(m_global):
        cardinality = np.random.randint(cardinality_low,cardinality_high)
        idx = np.random.choice(importance_nominal,p=weights,size=cardinality,replace=False)
        buf = np.random.randint(1,11,len(idx))
        MU[j,idx] = buf
        TAU[j] = sum(MU[j,idx])*cardinality/dimension_global
    
    for cut in cut_range:
        
        print('cut = '+str(cut))
        print(' ')
        
        TAU_cut = TAU*cut # param set
        
        for omega in omega_range:
            
            print('omega = '+str(omega))
        
            counter = 0

            for id_start in list(np.random.choice(np.arange(n_global),size=num_starts,replace=False)):
                counter += 1
                print(' ')
                print(' start| '+str(counter))
                print(' ')

                guar_ulo,eta_out_ulo,u_out_ulo = ULO_OPO(list_q_h=-PI,list_q_c=MU,list_kappa_h=PHI,list_kappa_c=-TAU_cut,omega=omega,M=M_true,tau=tau_def,init=id_start,rho=1e-3,domain_size=ds,extra_verb=verb_,\
            MAX_TIME=timing,\
            MAX_ITER=maxit,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF)

                list_RESULTS.append(guar_ulo)
                list_algorithm.append('ULO')
                list_prob.append(prob_id)
                list_rep.append(counter)
                list_omegas.append(omega)
                list_cuts.append(cut)

                guar_ram,eta_out_ram,u_out_ram = RAM_OPO(list_q_h=-PI,list_q_c=MU,list_kappa_h=PHI,list_kappa_c=-TAU_cut,omega=omega,M=M_true,tau=tau_def,init=id_start,rho=1e-3,domain_size=ds,extra_verb=verb_,\
            MAX_TIME=timing,\
            MAX_ITER=np.inf,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF)

                list_RESULTS.append(guar_ram)
                list_algorithm.append('RAM')
                list_prob.append(prob_id)
                list_rep.append(counter)
                list_omegas.append(omega)
                list_cuts.append(cut)

            time_val_tuples_es,eta_out_es,u_out_es = ES_OPO(list_q_h=-PI,list_q_c=MU,list_kappa_h=PHI,list_kappa_c=-TAU_cut,omega=omega,M=M_true,tau=tau_def,init=id_start,domain_size=ds,extra_verb=verb_,\
        MAX_TIME=np.inf,\
        MAX_ITER=np.inf,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF)

            list_RESULTS.append(time_val_tuples_es)
            list_algorithm.append('ES')
            list_prob.append(prob_id)
            list_rep.append(counter)
            list_omegas.append(omega)
            list_cuts.append(cut)
            
            BOUNDs,btypes,Ts,ALG,CUT,OMEGA,PROBID,REPID = [],[],[],[],[],[],[],[]
            for res,alg,pbid,repid,cu,om in zip(list_RESULTS,list_algorithm,list_prob,list_rep,list_cuts,list_omegas):
                for line in res:
                    if alg=='ES':
                        BOUNDs += [line[1],np.inf]
                    else:
                        BOUNDs += [line[1],line[2]]                    
                    btypes += ['UB','LB']
                    Ts += [line[0],line[0]]
                    ALG += [alg,alg]
                    CUT += [cu,cu]
                    OMEGA += [om,om]
                    PROBID += [pbid,pbid]
                    REPID += [repid,repid]
                    
            experiment_df = pd.DataFrame(data={'cut':CUT,'omega':OMEGA,'algo':ALG,'times':Ts,'bounds':BOUNDs,'bound_type':btypes,\
                                           'probID':PROBID,'repID':REPID})
            
            experiment_df.to_csv(name+'_nopta_closer_3.csv')
        
    print(' ')

cut = 1
 
omega = 0.5
 
 start| 1
 
=== NEW ULO SOLVE ===
 
-> i hat = 196
 
t = 0.7930037975311279
F_hat: -91.59069961360655 for i_bar = 196
|S_1| = 11
i hat = 460
t = 11.305048942565918
F_hat: -91.5907 & F_check: -148.335| GAP: 56.744315337081105 || target: 0
t = 11.740050077438354
F_hat: -95.53351570823693 for i_bar = 460
|S_2| = 24
i hat = 371
t = 22.11109447479248
F_hat: -95.5335 & F_check: -128.1227| GAP: 32.589208455258614 || target: 0
t = 22.573096990585327
F_hat: -98.84773166339437 for i_bar = 371
|S_3| = 37
i hat = 454
t = 33.29614329338074
F_hat: -98.8477 & F_check: -120.6175| GAP: 21.769764590590896 || target: 0
|S_4| = 50
i hat = 393
t = 44.96919226646423
F_hat: -98.8477 & F_check: -116.2918| GAP: 17.444065078238367 || target: 0
|S_5| = 63
i hat = 273
t = 56.68624305725098
F_hat: -98.8477 & F_check: -111.1947| GAP: 12.346991058552675 || target: 0
t = 57.12324380874634
F_hat: -101.49108437231929 for i_bar = 273
|S_6| = 76
i hat = 103
t = 68.24028992652893
F_hat: -101.4911 &

i hat = 495
t = 53.34367775917053
F_hat: -102.1202 & F_check: -109.4965| GAP: 7.376304254201941 || target: 0
|S_6| = 83
i hat = 455
t = 64.2810173034668
F_hat: -102.1202 & F_check: -107.4436| GAP: 5.323338916339694 || target: 0
|S_7| = 97
i hat = 27
t = 75.31531262397766
F_hat: -102.1202 & F_check: -104.9742| GAP: 2.8539791366643215 || target: 0
|S_8| = 105
i hat = 273
t = 86.35160493850708
F_hat: -102.1202 & F_check: -104.5217| GAP: 2.4015207494544484 || target: 0
|S_9| = 112
i hat = 341
t = 97.85791063308716
F_hat: -102.1202 & F_check: -103.6783| GAP: 1.5580263523967375 || target: 0
|S_10| = 120
i hat = 446
t = 109.19321179389954
F_hat: -102.1202 & F_check: -103.567| GAP: 1.4467690880812398 || target: 0
|S_11| = 131
i hat = 380
t = 121.01252603530884
F_hat: -102.1202 & F_check: -102.9688| GAP: 0.848603863687444 || target: 0
|S_12| = 141
i hat = 131
t = 132.77784085273743
F_hat: -102.1202 & F_check: -102.6227| GAP: 0.5024601961885793 || target: 0
|S_13| = 149
i hat = 144
t = 144.47915

t = 0.7330102920532227
F_hat: -81.45952291165705 for i_bar = 240
|S_1| = 17
i hat = 460
t = 10.665658235549927
F_hat: -81.4595 & F_check: -137.0494| GAP: 55.58983172028152 || target: 0
t = 11.1156644821167
F_hat: -95.53351570823693 for i_bar = 460
|S_2| = 31
i hat = 244
t = 21.09780478477478
F_hat: -95.5335 & F_check: -124.9624| GAP: 29.42889578908232 || target: 0
|S_3| = 45
i hat = 273
t = 31.44794797897339
F_hat: -95.5335 & F_check: -117.7693| GAP: 22.235806141694297 || target: 0
t = 31.856953382492065
F_hat: -101.49108437231929 for i_bar = 273
|S_4| = 61
i hat = 307
t = 42.260101079940796
F_hat: -101.4911 & F_check: -113.7738| GAP: 12.28273726870006 || target: 0
|S_5| = 77
i hat = 446
t = 53.12125086784363
F_hat: -101.4911 & F_check: -109.9995| GAP: 8.508453915227179 || target: 0
|S_6| = 87
i hat = 27
t = 64.21440601348877
F_hat: -101.4911 & F_check: -109.3592| GAP: 7.868119652662159 || target: 0
|S_7| = 99
i hat = 103
t = 75.63656520843506
F_hat: -101.4911 & F_check: -107.9491| GAP

|S_7| = 86
i hat = 451
t = 77.12556052207947
F_hat: -102.1202 & F_check: -108.8227| GAP: 6.7025033459055265 || target: 0
|S_8| = 93
i hat = 446
t = 88.2576413154602
F_hat: -102.1202 & F_check: -106.0855| GAP: 3.965235646963194 || target: 0
|S_9| = 104
i hat = 380
t = 99.30372142791748
F_hat: -102.1202 & F_check: -105.2494| GAP: 3.1291324420394915 || target: 0
|S_10| = 115
i hat = 414
t = 110.63180446624756
F_hat: -102.1202 & F_check: -104.4318| GAP: 2.3116008264611736 || target: 0
|S_11| = 124
i hat = 131
t = 121.71288537979126
F_hat: -102.1202 & F_check: -103.3202| GAP: 1.1999501701181856 || target: 0
|S_12| = 135
i hat = 144
t = 133.15896844863892
F_hat: -102.1202 & F_check: -102.1202| GAP: 0 || target: 0
 
=== total time: 133.15996980667114 ===
|H|/n = 0.026
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -102.12022534190585
 
=== NEW RAM SOLVE ===
 
-> i hat = 236
 
t = 1.2340087890625
F_hat <= -87.09997196258195 for i_bar = 372
t = 3.5210251808166504
F_hat <= -89.451937122

t = 0.7980031967163086
F_hat <= -64.2801171016734 for i_bar = 110
t = 1.2540061473846436
F_hat <= -71.1870061693139 for i_bar = 357
t = 2.1810083389282227
F_hat <= -95.53351570823693 for i_bar = 460
t = 5.226019859313965
F_hat <= -98.79958414374418 for i_bar = 393
t = 17.111064672470093
F_hat <= -101.49108437231929 for i_bar = 273
t = 21.452082872390747
F_hat <= -102.12022534190585 for i_bar = 144
 
=== total time: 225.23615527153015 ===
|H|/n = 1.0
 -> final F = -102.12022534190585
 
 
 start| 15
 
=== NEW ULO SOLVE ===
 
-> i hat = 491
 
t = 1.3290431499481201
F_hat: -95.06239302975476 for i_bar = 452
|S_1| = 16
i hat = 371
t = 11.443369626998901
F_hat: -95.0624 & F_check: -140.9402| GAP: 45.87783519479672 || target: 0
t = 11.932385206222534
F_hat: -98.84773166339437 for i_bar = 371
|S_2| = 31
i hat = 322
t = 22.148731231689453
F_hat: -98.8477 & F_check: -121.7211| GAP: 22.873375330550076 || target: 0
|S_3| = 49
i hat = 14
t = 32.874077796936035
F_hat: -98.8477 & F_check: -119.3921| 

i hat = 322
t = 10.410151720046997
F_hat: -119.8976 & F_check: -167.9439| GAP: 48.046338891211036 || target: 0
t = 10.831157445907593
F_hat: -153.67399283657085 for i_bar = 322
|S_2| = 13
i hat = 460
t = 20.455296277999878
F_hat: -153.674 & F_check: -159.6968| GAP: 6.022762645974495 || target: 0
t = 20.872302055358887
F_hat: -157.55766015283893 for i_bar = 460
|S_3| = 17
i hat = 460
t = 30.737445831298828
F_hat: -157.5577 & F_check: -157.5577| GAP: 0 || target: 0
 
=== total time: 30.737445831298828 ===
|H|/n = 0.006
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -157.55766015283893
 
=== NEW RAM SOLVE ===
 
-> i hat = 405
 
t = 0.7590115070343018
F_hat <= -119.89758252578895 for i_bar = 405
t = 1.1880168914794922
F_hat <= -141.52452475360906 for i_bar = 6
t = 3.872056245803833
F_hat <= -145.48995546961598 for i_bar = 255
t = 5.163074731826782
F_hat <= -145.94386550110872 for i_bar = 371
t = 9.325134038925171
F_hat <= -151.21020640580832 for i_bar = 239
t = 20.8923020362854
F_

t = 10.078065872192383
F_hat: -153.67399283657085 for i_bar = 322
|S_2| = 9
i hat = 460
t = 19.089123249053955
F_hat: -153.674 & F_check: -160.6585| GAP: 6.984555969784509 || target: 0
t = 19.586126565933228
F_hat: -157.55766015283893 for i_bar = 460
|S_3| = 13
i hat = 6
t = 29.645193099975586
F_hat: -157.5577 & F_check: -159.7827| GAP: 2.2250171945428576 || target: 0
|S_4| = 20
i hat = 468
t = 40.19725966453552
F_hat: -157.5577 & F_check: -158.0003| GAP: 0.4426319937383596 || target: 0
|S_5| = 27
i hat = 460
t = 50.297330141067505
F_hat: -157.5577 & F_check: -157.5577| GAP: 0 || target: 0
 
=== total time: 50.297330141067505 ===
|H|/n = 0.01
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -157.55766015283893
 
=== NEW RAM SOLVE ===
 
-> i hat = 363
 
t = 0.7280032634735107
F_hat <= -95.89956019510517 for i_bar = 363
t = 2.01001238822937
F_hat <= -110.6282265744654 for i_bar = 166
t = 2.96901798248291
F_hat <= -151.21020640580832 for i_bar = 239
t = 44.953289270401
F_hat <= -15

|S_8| = 128
i hat = 307
t = 103.27475166320801
F_hat: -68.8502 & F_check: -68.8502| GAP: 0 || target: 0
 
=== total time: 103.27475166320801 ===
|H|/n = 0.02
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -68.85018411499085
 
=== NEW RAM SOLVE ===
 
-> i hat = 363
 
t = 1.3280308246612549
F_hat <= -68.85018411499085 for i_bar = 307
 
=== total time: 240.2802631855011 ===
|H|/n = 0.928
/!\ EXIT: search time over... /!\ 
 -> final F = -68.85018411499085
 
 
 start| 2
 
=== NEW ULO SOLVE ===
 
-> i hat = 230
 
t = 1.3230314254760742
F_hat: -68.85018411499085 for i_bar = 307
|S_1| = 22
i hat = 380
t = 11.76628065109253
F_hat: -68.8502 & F_check: -97.5091| GAP: 28.65887645676412 || target: 0
|S_2| = 36
i hat = 107
t = 23.54756188392639
F_hat: -68.8502 & F_check: -93.8266| GAP: 24.97638292277007 || target: 0
|S_3| = 63
i hat = 429
t = 35.567816734313965
F_hat: -68.8502 & F_check: -76.9895| GAP: 8.139342600569094 || target: 0
|S_4| = 72
i hat = 438
t = 48.337077140808105
F_hat: -68.8

|S_8| = 128
i hat = 452
t = 98.94406604766846
F_hat: -68.8502 & F_check: -70.3951| GAP: 1.5449369155248291 || target: 0
|S_9| = 134
i hat = 307
t = 112.93119788169861
F_hat: -68.8502 & F_check: -68.8502| GAP: 0 || target: 0
 
=== total time: 112.93119788169861 ===
|H|/n = 0.026
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -68.85018411499085
 
=== NEW RAM SOLVE ===
 
-> i hat = 228
 
t = 1.1770110130310059
F_hat <= -66.1768306276029 for i_bar = 225
t = 2.5920238494873047
F_hat <= -68.33633328569738 for i_bar = 30
t = 4.519040822982788
F_hat <= -68.85018411499085 for i_bar = 307
 
=== total time: 240.46033477783203 ===
|H|/n = 0.92
/!\ EXIT: search time over... /!\ 
 -> final F = -68.85018411499085
 
 
 start| 7
 
=== NEW ULO SOLVE ===
 
-> i hat = 471
 
t = 1.1810111999511719
F_hat: -68.33633328569738 for i_bar = 30
|S_1| = 21
i hat = 341
t = 13.051120519638062
F_hat: -68.3363 & F_check: -107.7912| GAP: 39.454916577068786 || target: 0
|S_2| = 39
i hat = 307
t = 24.49422550201

|S_8| = 121
i hat = 307
t = 99.69489979743958
F_hat: -68.8502 & F_check: -68.8502| GAP: 0 || target: 0
 
=== total time: 99.69489979743958 ===
|H|/n = 0.022
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -68.85018411499085
 
=== NEW RAM SOLVE ===
 
-> i hat = 334
 
t = 1.2470040321350098
F_hat <= -67.24229718070318 for i_bar = 452
t = 3.0670111179351807
F_hat <= -67.24900731391749 for i_bar = 121
t = 4.682016372680664
F_hat <= -68.85018411499085 for i_bar = 307
 
=== total time: 240.14915323257446 ===
|H|/n = 0.918
/!\ EXIT: search time over... /!\ 
 -> final F = -68.85018411499085
 
 
 start| 12
 
=== NEW ULO SOLVE ===
 
-> i hat = 50
 
t = 1.2770044803619385
F_hat: -61.053656459135944 for i_bar = 55
|S_1| = 20
i hat = 61
t = 11.25504183769226
F_hat: -61.0537 & F_check: -105.2244| GAP: 44.17074369001748 || target: 0
t = 11.660043001174927
F_hat: -65.37379070941364 for i_bar = 61
|S_2| = 32
i hat = 273
t = 22.563081979751587
F_hat: -65.3738 & F_check: -101.955| GAP: 36.5811594